### ライブラリ読み込み & Client 初期化

In [ ]:
import os
from dotenv import load_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    MemoryStoreDefaultDefinition,
    MemoryStoreDefaultOptions,
    ResponsesUserMessageItemParam,
    MemorySearchOptions
)

load_dotenv()

endpoint = os.environ["project_endpoint"]

# Foundry (Azure AI Projects) client 初期化
client = AIProjectClient(
    endpoint=endpoint,
    credential=DefaultAzureCredential()
)

print("Foundry Client 初期化完了")

### メモリストアを作る

In [ ]:
memory_store_name = "my_memory_store"

# メモリストアの設定
definition = MemoryStoreDefaultDefinition(
    chat_model="gpt-4.1",                     
    embedding_model="text-embedding-3-small", 
    options=MemoryStoreDefaultOptions(
        user_profile_enabled=True,
        chat_summary_enabled=True
        # user_profile_details=(
    #     "Focus on coffee preferences (roast level, milk/sugar, time of day). "
    #     "Avoid storing sensitive data such as exact address or financial information."
    # ),  # （任意）どんなプロファイルを重視し、何を避けるかを文章で指示できる
    )
)

# メモリストア作成
memory_store = client.memory_stores.create(
    name=memory_store_name,
    definition=definition,
    description="ワークショップ用メモリストア",
)

print(f"✅ メモリストア作成完了: {memory_store.name}")


### メモリを追加（更新）

In [ ]:
# scope は “ユーザーごとのメモリ領域”
scope = "user_123"

# メモリとして保存したい内容
user_message = ResponsesUserMessageItemParam(
    content="私は濃いめのコーヒーが好きで、朝に飲むことが多いです。"
)

update_poller = client.memory_stores.begin_update_memories(
    name=memory_store_name,
    scope=scope,
    items=[user_message],
    update_delay=0,  # すぐ実行。まとめて複数ターンの会話を受けてからメモリ抽出したい」場合の待ち時間を指定できる。
)

update_result = update_poller.result()

print(f"✅ メモリ追加完了: {len(update_result.memory_operations)} 件")
for op in update_result.memory_operations:
    print(f" - {op.kind}: {op.memory_item.content}")


### メモリを検索する（取り出す）

In [ ]:
query = ResponsesUserMessageItemParam(
    content="私のコーヒーの好みは？"
)

search_response = client.memory_stores.search_memories(
    name=memory_store_name,
    scope=scope,
    items=[query],
    options=MemorySearchOptions(max_memories=5)
)

print(f"🔍 見つかったメモリ数: {len(search_response.memories)}")
for m in search_response.memories:
    print(f" - {m.memory_item.memory_id}: {m.memory_item.content}")


### メモリストアの一覧表示

In [ ]:
stores = client.memory_stores.list()

print("📦 メモリストア一覧:")
for store in stores:   
    print(f"- {store.name} ({store.description})")


### scope のメモリ削除（ユーザー別データ削除）

In [ ]:
client.memory_stores.delete_scope(
    name=memory_store_name,
    scope=scope,
)

print(f"🗑 scope `{scope}` のメモリを削除しました")


# Foundry Portal で メモリを利用してる方はこちらから

In [ ]:
import os
from dotenv import load_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    MemoryStoreDefaultDefinition,
    MemoryStoreDefaultOptions,
    ResponsesUserMessageItemParam,
    MemorySearchOptions
)

load_dotenv()

endpoint = os.environ["project_endpoint"]

# Foundry (Azure AI Projects) client 初期化
client = AIProjectClient(
    endpoint=endpoint,
    credential=DefaultAzureCredential()
)

print("Foundry Client 初期化完了")

### メモリストアの一覧表示

In [ ]:
stores = client.memory_stores.list()

print("📦 メモリストア一覧:")
for store in stores:   
    print(f"- {store.name} ({store.description})")

### 既存のメモリを見たいとき

In [ ]:
memory_store_name = "MemoryStore-XXXX" # 既存のメモリストア名を指定。通常 MemoryStore-xxxxxx の形式。
scope = "defaultUser" # Foundryでは通常 defaulUser と設定されていることが多い。

search_response = client.memory_stores.search_memories(
    name=memory_store_name,
    scope=scope,
    items=[],
    options=MemorySearchOptions(max_memories=5)
)

print(f"🔍 見つかったメモリ数: {len(search_response.memories)}")
for m in search_response.memories:
    print(f" - {m.memory_item.memory_id}: {m.memory_item.content}")